In [13]:
import re
from bs4 import BeautifulSoup

In [288]:
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from time import sleep

def download(url, params={}, headers={}, method='GET', maximum=2):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [11]:
# 최대한 다양한 사이트들을 크롤링해보면
# 사이트의 구조에 익숙해질 수 있음
url = 'http://www.pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [14]:
dom.find_all('img', attrs={'src':re.compile(r'\d\.jpg$')})

[<img src="../img/gifts/img1.jpg"/>,
 <img src="../img/gifts/img2.jpg"/>,
 <img src="../img/gifts/img3.jpg"/>,
 <img src="../img/gifts/img4.jpg"/>,
 <img src="../img/gifts/img6.jpg"/>]

In [16]:
# $111,111,111  (\d+ (숫자가 한번이상 반복되다가) [,])* (이게 0번이상 반복) \d+([.]\d+)? (소숫점 아래 2글자)
# $111
# $111,110.00
# $111.00
dom.find_all(text=re.compile(r'\$(\d+[,])*\d+(?:[.]\d+)?\b$'))

['\n$15.00\n', '\n$10,000.52\n', '\n$10,005.00\n', '\n$0.50\n', '\n$1.50\n']

In [289]:
# Google 검색 결과 예시
url = 'https://www.google.com/search'
params = {
    'q' : ['파이썬'],
    'hl': ['ko'],
    'sxsrf': ['ALeKk00P1KJuqgmTCupZG-YsxVpVk3mRGw:1625639013354'],
    'source': ['hp'],
    'ei': ['ZUjlYJa3CsWE-gSfoJqoBA'],
    'iflsig': ['AINFCbYAAAAAYOVWdT9ryM-9TZT3LtSw2ZwMuAiXXfsN'],
    'gs_lcp': ['Cgdnd3Mtd2l6EAEYATIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJ1AAWABg8ckFaAFwAHgAgAEAiAEAkgEAmAEAqgEHZ3dzLXdperABCg'],
    'sclient': ['gws-wiz']
}

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

params['q'] = '파이썬'

resp = download(url, params, headers, 'GET')

In [51]:
resp.request.url

'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&sxsrf=ALeKk00P1KJuqgmTCupZG-YsxVpVk3mRGw%3A1625639013354&source=hp&ei=ZUjlYJa3CsWE-gSfoJqoBA&iflsig=AINFCbYAAAAAYOVWdT9ryM-9TZT3LtSw2ZwMuAiXXfsN&gs_lcp=Cgdnd3Mtd2l6EAEYATIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJ1AAWABg8ckFaAFwAHgAgAEAiAEAkgEAmAEAqgEHZ3dzLXdperABCg&sclient=gws-wiz'

In [19]:
# HTML의 class : OOP의 class와 비슷. (다중상속 가능, 여러 개일수 있음, ...)
# HTML의 id : 단 1개만 있음!

dom = BeautifulSoup(resp.text, 'lxml')

In [50]:
result = dom.find(attrs={'id':'rso'})
res_list = result.find_all(recursive=False)
for _ in res_list:
#     print(_.find_all(recursive=False, limit=2)[-1])
    print(_.find('a').find('h3').text)
    print(_.find('a').attrs['href'])
#     break
# [(_.name, _['class']) for _ in result.find_all(recursive=False)] # 검색 결과가 9개.

파이썬 - 위키백과, 우리 모두의 백과사전
https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
파이썬 자습서 — Python 3.9.6 문서
https://docs.python.org/ko/3/tutorial/index.html
Python - 나무위키
https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python: 쉽게 풀어 쓴 기초 ...
https://wikidocs.net/43
Python - 파이썬이란? - 코딩의 시작, TCP School
http://tcpschool.com/python/intro


AttributeError: 'NoneType' object has no attribute 'text'

In [48]:
# dom.find_all('h3', attrs={'class':'LC20lb DKV0Md'})
for _ in dom.find_all('h3', attrs={'class':'LC20lb DKV0Md'}):
    print(_.text)
    print(_.find_parent().attrs['href'])

파이썬 - 위키백과, 우리 모두의 백과사전
https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
파이썬 자습서 — Python 3.9.6 문서
https://docs.python.org/ko/3/tutorial/index.html
Python - 나무위키
https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python: 쉽게 풀어 쓴 기초 ...
https://wikidocs.net/43
Python - 파이썬이란? - 코딩의 시작, TCP School
http://tcpschool.com/python/intro
모두를 위한 프로그래밍 : 파이썬 강좌소개 : edwith
https://www.edwith.org/pythonforeverybody
1 장 파이썬(Python) 입문 | 파이썬 프로그래밍 기초
http://bigdata.dongguk.ac.kr/lectures/Python/_book/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9E%85%EB%AC%B8.html
파이썬 입문 | 프로그래머스
https://programmers.co.kr/learn/courses/2


---

In [290]:
# 네이버 검색
url = 'https://search.naver.com/search.naver'
params = {
    'where': ['nexearch'],
    'sm': ['top_hty'],
    'fbm': ['0'],
    'ie': ['utf8'],
    'query': ['통영여행']
}

resp = download(url, params, headers, 'GET')

In [55]:
dom = BeautifulSoup(resp.text, 'lxml')

In [60]:
for _ in dom.find_all('a', attrs={'class':'api_txt_lines'}):
    print(_.text.strip())
    print(_.attrs['href'])

통영 여행 놀거리 아라마린서비스 요트투어
https://blog.naver.com/obh1680/222398983097
거제 통영 여행 외도 포함 내품 핫플 코스
https://blog.naver.com/lightsout/222288330327
통영 여행코스 갈만한곳 이순신공원 바다 전망
https://blog.naver.com/suk4408/222338705742
거제 통영 여행 코스 서호시장부터 동피랑벽화마을까지
https://blog.naver.com/bada3347/222360711590
통영 여행코스 이순신공원, 통영 서피랑마을까지
https://blog.naver.com/hnmh2332/222313684322
국내 1박2일 통영 여행 디피랑 서피랑 동피랑 벽화마을 이순신공원
https://blog.naver.com/hloveh01/222386234715
거제 통영 여행 코스 통영 이순신공원 등 1박2일 여행지
https://blog.naver.com/ultrabat/222362354570
통영 가볼만한곳 이순신공원 통영 여행코스
https://blog.naver.com/jyleen/222342610447
아이와 비오는 통영여행 3박 4일 코스
https://blog.naver.com/sidna/222358406013
여름에 통영여행 일정 좀 봐주세요 여름 휴가로 통영 여행 가려고 하는데요 너무 더울 것 같아 동피랑 같은 언덕으로 많이 걸어야하는 여행지는 패스하고 추천 좀 해주세요!! 3박 4일로 있을거에요,,
https://kin.naver.com/qna/detail.nhn?d1id=9&dirId=90111&docId=393379306&qb=7Ya17JiB7Jes7ZaJ&enc=utf8&section=kin.ext&rank=1&search_sort=0&spq=0
통영 여행을 계획하고 계시다면 통영 케이블카와 스카이라인 루지, 이순신공원, 욕지도를 추천드립니다. 풍경이 아름다운 곳들도 많고 맛집들도 많다고 합니다. 자세한 내용이 담겨 있는 링크도 같이

In [78]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

# can_fetch(url, '/')
can_fetch('https://www.google.com', '/search')

(False,
 [],
 ['/search',
  '/search/about',
  '/search/static',
  '/search/howsearchworks',
  '/sdch',
  '/groups',
  '/index.html?',
  '/?',
  '/?hl=',
  '/?hl=*&',
  '/?hl=*&gws_rd=ssl$',
  '/?hl=*&*&gws_rd=ssl',
  '/?gws_rd=ssl$',
  '/?pt1=true$',
  '/imgres',
  '/u/',
  '/preferences',
  '/setprefs',
  '/default',
  '/m?',
  '/m/',
  '/m/finance',
  '/wml?',
  '/wml/?',
  '/wml/search?',
  '/xhtml?',
  '/xhtml/?',
  '/xhtml/search?',
  '/xml?',
  '/imode?',
  '/imode/?',
  '/imode/search?',
  '/jsky?',
  '/jsky/?',
  '/jsky/search?',
  '/pda?',
  '/pda/?',
  '/pda/search?',
  '/sprint_xhtml',
  '/sprint_wml',
  '/pqa',
  '/palm',
  '/gwt/',
  '/purchases',
  '/local?',
  '/local_url',
  '/shihui?',
  '/shihui/',
  '/products?',
  '/product_',
  '/products_',
  '/products;',
  '/print',
  '/books/',
  '/bkshp?*q=*',
  '/books?*q=*',
  '/books?*output=*',
  '/books?*pg=*',
  '/books?*jtp=*',
  '/books?*jscmd=*',
  '/books?*buy=*',
  '/books?*zoom=*',
  '/books/about',
  '/booksright

---

In [88]:
# 공공데이터 API 활용하기
# 이 데이터는 XML 방식밖에 지원 안함.
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
params = {
    'serviceKey':'ibPhO0aMSaZiYiGNKqQ+GntOVhqamQs3+pNeSYjVz3OSX1RnIKShl/7jibL7YY7t3KPLvrZYk48OrEhLYFN59A==',
    'pageNo':1,
    'numOfRows':10,
    'startCreateDt':'20200310',
    'endCreateDt':'20200315'
}
resp = download(url, params=params) # header 필요없음.. API 사용하므로

# 실제 API 사용 시 사용 설명서 (빽서)를 다 읽어보고 사용해야 함.

In [89]:
resp.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><accDefRate>3.2396602365</accDefRate><accExamCnt>268212</accExamCnt><accExamCompCnt>251940</accExamCompCnt><careCnt>7300</careCnt><clearCnt>834</clearCnt><createDt>2020-03-15 10:01:22.22</createDt><deathCnt>75</deathCnt><decideCnt>8162</decideCnt><examCnt>16272</examCnt><resutlNegCnt>243778</resutlNegCnt><seq>74</seq><stateDt>20200315</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-15 10:01:22.22</updateDt></item><item><accDefRate>3.3180003364</accDefRate><accExamCnt>261335</accExamCnt><accExamCompCnt>243701</accExamCompCnt><careCnt>7402</careCnt><clearCnt>714</clearCnt><createDt>2020-03-14 10:29:11.11</createDt><deathCnt>72</deathCnt><decideCnt>8086</decideCnt><examCnt>17634</examCnt><resutlNegCnt>235615</resutlNegCnt><seq>73</seq><stateDt>20200314</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-14 10:29:11

In [90]:
# HTML Language가 아닌 XML인데도 불구하고
# 어짜피 태그로 구성된 Markup Language이므로 마찬가지로
# BeautifulSoup을 사용해서 parsing 진행하면 됨.
dom = BeautifulSoup(resp.text, 'lxml')

In [96]:
len(dom.find_all('item'))

6

In [97]:
[_.name for _ in dom.find_all('item')[0].find_all(recursive=False)]

['accdefrate',
 'accexamcnt',
 'accexamcompcnt',
 'carecnt',
 'clearcnt',
 'createdt',
 'deathcnt',
 'decidecnt',
 'examcnt',
 'resutlnegcnt',
 'seq',
 'statedt',
 'statetime',
 'updatedt']

In [99]:
for _ in dom.find_all('item')[0].find_all(recursive=False):
    print(_.name)
    print(_.text.strip())
    print()

accdefrate
3.2396602365

accexamcnt
268212

accexamcompcnt
251940

carecnt
7300

clearcnt
834

createdt
2020-03-15 10:01:22.22

deathcnt
75

decidecnt
8162

examcnt
16272

resutlnegcnt
243778

seq
74

statedt
20200315

statetime
00:00

updatedt
2020-03-15 10:01:22.22



2주차 수업 내용 : 데이터 수집하는 방법!

  정형,        반정형(문서),         비정형
DB(RDB-ORM), HTML-XML-JSON-(CSV),  텍스트-이미지
sqlite,alchemy,   beautifulsoup, request, re -> HTTP
              DOM(구조적으로 만들어짐-상하관계),            parser(html.parser, lxml-안정적,빠른, html5lib-브라우저)
     * HTML은 not well-formed 문서이므로 실제 제작된 DOM이
       다른 모습일 수 있다.
\* 불법 조심 : robots.txt, 출처 Opt-out형태로 요청 있을 시 삭제, 그것도 없으면 API 사용.

=> Cralwer(Scraping) : 빅데이터 수집기

In [ ]:
# CSSOM
# DOM을 만들 때 CSS를 사용할 수 있음. Object 모델이므로 ... 
# selector : 특정 태그, 특정 ...을 선택.

# 다른 곳에서 다 쓰일 수 있음. (JS, CSS, HTML, ...)
# HTML -> DOM Tree (오브젝드 모델)
# CSS -> CSS Parser -> CSS Object 파일
# 같은 파일이었지만, 서로 바라보는 방식이 다름. 둘을 합쳐버림.
# => DOM, CSSOM, Render Tree

# CSS의 Selector -> h1.
# CSSOM -> 약간의 구조적/관계적이 존재. DOM보다 표현력이 훨씬 더 높음.

In [100]:
url = 'http://www.pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [104]:
# CSSOM에서 Selector 내용
 # nth-of-type만 지원?
dom.select('td') # td를 쓰면 온갖 종류의 열을 찾게 됨.
dom.select('td, tr') # td와 tr을 모두 찾게 됨.
dom.select('tr td') # ** tr의 자손 중에 td를 찾으라는 뜻. 즉, find(recursive=True)
dom.select('tr>td') # ** tr의 자손 중에 td만을 찾으라는 뜻. find(recursive=False)

          1열    2열     3열  ....
1행<tr>
2행<tr>
3행<tr>

select('tr') = 3개
select('td') = 행*열 [1,2,3,4,5,...]
select('tr td') = (자손)행*열 [1,2,3,4,5,...]
select('tr > td') = (자식)행*열 [1,2,3,4,5,...]
select('tr > td:nth-of-type(2)') = 1행2열, 2행2열, 3행2열, ...

[<td>
 Vegetable Basket
 </td>,
 <td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td>,
 <td>
 $15.00
 </td>,
 <td>
 <img src="../img/gifts/img1.jpg"/>
 </td>,
 <td>
 Russian Nesting Dolls
 </td>,
 <td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td>,
 <td>
 $10,000.52
 </td>,
 <td>
 <img src="../img/gifts/img2.jpg"/>
 </td>,
 <td>
 Fish Painting
 </td>,
 <td>
 If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
 </td>,
 <td>
 $10,005.00
 </td>,
 <td>
 <img src="../img/gifts/img3.jpg"/>
 </td>,
 <td>
 Dead Parrot
 </td>,
 <td>
 This is an ex-parrot! <span class="excitingNote">Or maybe he's only resting?</span>
 </t

In [107]:
[_.text.strip() for _ in dom.select('tr > td:nth-of-type(3)')]

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']

In [ ]:
* 중요
tr > td:nth of type 각 행에서 자식 열의 몇 번째
     td:nth of type 모든 열에서 몇 번째
tr td:nth of type   각 행의 모든 열의 몇 번째

In [112]:
# dom.select('tr > td:nth-of-type(4) > img')
# dom.select('tr > td:nth-of-type(4) img[src$=jpg]')
dom.select('tr > td:nth-of-type(4) img[src^=..]')   # ??

SelectorSyntaxError: Malformed attribute selector at position 26
  line 1:
tr > td:nth-of-type(4) img[src^=..]
                          ^

In [116]:
len(dom.select('#gift1')), dom.select('.gift')

(1,
 [<tr class="gift" id="gift1"><td>
  Vegetable Basket
  </td><td>
  This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
  <span class="excitingNote">Now with super-colorful bell peppers!</span>
  </td><td>
  $15.00
  </td><td>
  <img src="../img/gifts/img1.jpg"/>
  </td></tr>,
  <tr class="gift" id="gift2"><td>
  Russian Nesting Dolls
  </td><td>
  Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
  </td><td>
  $10,000.52
  </td><td>
  <img src="../img/gifts/img2.jpg"/>
  </td></tr>,
  <tr class="gift" id="gift3"><td>
  Fish Painting
  </td><td>
  If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
  </td><td>
  $10,005.00
  </td><td>
  <img src="../img/gifts/img3.jpg"/>
  </td></tr>,
  <tr class="gift"

In [ ]:
# 앞선 예시들이 이 selector을 사용하면 얼마나 간단해지는 지!

In [291]:
url = 'https://www.google.com/search'
params = {
    'q' : ['파이썬'],
    'hl': ['ko'],
    'sxsrf': ['ALeKk00P1KJuqgmTCupZG-YsxVpVk3mRGw:1625639013354'],
    'source': ['hp'],
    'ei': ['ZUjlYJa3CsWE-gSfoJqoBA'],
    'iflsig': ['AINFCbYAAAAAYOVWdT9ryM-9TZT3LtSw2ZwMuAiXXfsN'],
    'gs_lcp': ['Cgdnd3Mtd2l6EAEYATIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJ1AAWABg8ckFaAFwAHgAgAEAiAEAkgEAmAEAqgEHZ3dzLXdperABCg'],
    'sclient': ['gws-wiz']
}

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

params['q'] = '파이썬'

resp = download(url, params, headers, 'GET')

In [118]:
dom = BeautifulSoup(resp.text, 'html5lib')

In [121]:
len(dom.select('h3')), len(dom.select('h3.LC20lb'))\
, len(dom.select('h3.LC20lb.DKV0Md'))

(12, 8, 8)

In [122]:
# 이건 대신 find 함수와 다르게 부모로 올라갈 수 없음.
# 대신 이웃한 애들은 찾을 수 있다.
len(dom.select('br + h3'))

8

In [126]:
dom.select('br + h3')[0].find_parent()

<a data-ved="2ahUKEwjT6-W7udLxAhU_GDQIHdWEDBwQFjAAegQIBRAD" href="https://docs.python.org/ko/3/tutorial/index.html" ping="/url?sa=t&amp;source=web&amp;rct=j&amp;url=https://docs.python.org/ko/3/tutorial/index.html&amp;ved=2ahUKEwjT6-W7udLxAhU_GDQIHdWEDBwQFjAAegQIBRAD"><br/><h3 class="LC20lb DKV0Md">파이썬 자습서 — Python 3.9.6 문서</h3><div class="TbwUpd NJjxre"><cite class="iUh30 Zu0yb qLRx3b tjvcx">https://docs.python.org<span class="dyjrff qzEoUe"> › tutorial</span></cite></div></a>

In [127]:
# CSS 문법(Selector)을 사용하면 훨씬 더 간단해짐.
for _ in dom.select('br + h3'):
    print(_.text.strip())
    print(_.find_parent().attrs['href'])

파이썬 자습서 — Python 3.9.6 문서
https://docs.python.org/ko/3/tutorial/index.html
파이썬 - 위키백과, 우리 모두의 백과사전
https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
Python - 나무위키
https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python: 쉽게 풀어 쓴 기초 ...
https://wikidocs.net/43
Python - 파이썬이란? - 코딩의 시작, TCP School
http://tcpschool.com/python/intro
모두를 위한 프로그래밍 : 파이썬 강좌소개 : edwith
https://www.edwith.org/pythonforeverybody
1 장 파이썬(Python) 입문 | 파이썬 프로그래밍 기초
http://bigdata.dongguk.ac.kr/lectures/Python/_book/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9E%85%EB%AC%B8.html
파이썬을 파이썬답게 | 프로그래머스
https://programmers.co.kr/learn/courses/4008


In [292]:
# 다음 예시인 네이버..
url = 'https://search.naver.com/search.naver'
params = {
    'where': ['nexearch'],
    'sm': ['top_hty'],
    'fbm': ['0'],
    'ie': ['utf8'],
    'query': ['통영여행']
}

resp = download(url, params, headers, 'GET')

In [130]:
dom = BeautifulSoup(resp.text, 'html5lib')

In [133]:
for _ in dom.select('.api_txt_lines.total_tit'):
    print(_.text.strip())
    print(_.attrs['href'])

통영 여행 놀거리 아라마린서비스 요트투어
https://blog.naver.com/obh1680/222398983097
거제 통영 여행 외도 포함 내품 핫플 코스
https://blog.naver.com/lightsout/222288330327
통영 여행코스 갈만한곳 이순신공원 바다 전망
https://blog.naver.com/suk4408/222338705742
거제 통영 여행 코스 서호시장부터 동피랑벽화마을까지
https://blog.naver.com/bada3347/222360711590
통영 여행코스 이순신공원, 통영 서피랑마을까지
https://blog.naver.com/hnmh2332/222313684322
국내 1박2일 통영 여행 디피랑 서피랑 동피랑 벽화마을 이순신공원
https://blog.naver.com/hloveh01/222386234715
거제 통영 여행 코스 통영 이순신공원 등 1박2일 여행지
https://blog.naver.com/ultrabat/222362354570
통영 가볼만한곳 이순신공원 통영 여행코스
https://blog.naver.com/jyleen/222342610447
아이와 비오는 통영여행 3박 4일 코스
https://blog.naver.com/sidna/222358406013


#### Scraping

---
### Crawler
잘 잠. 

In [293]:
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from time import sleep

def download(url, params={}, headers={}, method='GET', maximum=1):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [189]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

In [294]:
# 큐로 push, pop 구현할 거
def new_download(url):
    _ = urlparse(url)
    print(can_fetch(_[0]+'://'+_[1], _[2])[0])
    
    return download(url, headers=headers)

In [191]:
Queue = list() # 주소 값 들어가있을 거
seed = 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&sxsrf=ALeKk01gdzjZRape3rH9aLVuHvafXDcAhA%3A1625713882253&source=hp&ei=2mzmYMaADYnYz7sPir-w4AQ&iflsig=AINFCbYAAAAAYOZ66p1XqhEgkX96vWn92yq037Si3Vwk&oq=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gs_lcp=Cgdnd3Mtd2l6EAMyBAgjECcyBAgjECcyBAgjECcyBQgAELEDMgUIABCxAzIFCAAQsQMyBQgAELEDMgUIABCxAzIFCAAQsQMyBQgAELEDOggIABCxAxCDAToCCABQsgRYyQlgtwpoAnAAeAKAAXeIAfgFkgEDMC43mAEAoAEBqgEHZ3dzLXdpeg&sclient=gws-wiz&ved=0ahUKEwjG1dKTwNLxAhUJ7HMBHYofDEwQ4dUDCAc&uact=5'

new_download(seed)

False


In [158]:
dom = BeautifulSoup(resp.text, 'lxml')

In [159]:
len(dom.select('a'))

114

In [161]:
# 링크가 항상 a href로 걸리는 것이 아니라 내부적인 JS로 돌아갈 수도 있고, 다른 경우가 있을 수 있음.
_.has_attr('href'), len(dom.select('a[href]'))

# 이게 필터링이 훨씬 더 잘 될거임.

(True, 107)

In [162]:
links = [_.attrs['href'] for _ in dom.select('a[href]')]

In [164]:
links
# 이렇게 가져와버리면 절대 주소 및 상대 주소가 뒤죽박죽 섞여버림.. 
# 상대 주소의 경우는 앞에 domain을 붙여줘야만 함.
# # : 아무 역할 안함. 

# 이런 걸 제대로 처리해줘야만 한다..

['/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&gbv=1&sei=fm3mYOCZGJX49AOviKCYAw',
 'https://support.google.com/websearch/answer/181196?hl=ko',
 'https://www.google.com/webhp?hl=ko&sa=X&ved=0ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQPAgI',
 '#',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoAXoECAEQAw',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=vid&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoAnoECAEQBA',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=bks&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoA3oECAEQBQ',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoBHoECAEQBg',
 '/preferences',
 'https://docs.python.org/ko/3/tutorial/index.html',
 '#',
 'https://webcache.googleusercontent.com/search?q=cache:Rib_O5KvImcJ:https://docs.python.org/ko/3/tutorial/index.html+&cd=1&hl=ko&ct=clnk&gl=kr',
 'https://docs.python.org/ko/3/tutorial/int

In [169]:
links = [urljoin(seed, _.attrs['href']) for _ in dom.select('a[href]')
        if _.attrs['href'] != '#']

In [170]:
len(links)

94

In [295]:
directory = {'text':'TXT', 'application':'DOC', 'image':'IMG'}


def crawling(seed):
    resp = download(seed)
    #### 추가작업 -> Content-type: text/*, application/json+xml, image
    # 이런 다양한 데이터 타입때문에 이 크롤러는 죽을 것임. 수정 필요
    ctype = resp.headers['Content-type'].split('/')
    
    returnlist = list()
    
    if ctype[0] in ['text', 'application', 'image']:
        if ctype[0] == 'text':
            dom = BeautifulSoup(resp.text, 'lxml')


            for _ in dom.select('a[href], img[src], iframe[src]'):
                if _.has_attr('href'):
                    if not _.attrs['href'].startswith('#'):
                        returnlist.append(urljoin(seed, _.attrs['href']))
                else:
                    returnlist.append(urljoin(seed, _.attrs['src']))

            return returnlist

        path = directory[ctype[0]]
        path += '/'+seed.split('/')[-1]+'.'+ctype[-1]
#         re.sub(r'?\\', '', path) # 쓸모없는 문자들을 다 날려버림.
        
        with open(path, 'wb') as f:
            f.write(resp.content)
                
    return None
    
#     return [urljoin(seed, _.attrs['href'])
#              for _ in dom.select('a[href]') 
#              if not _.attrs['href'].startswith('#')] #원래는 검사를 실제로 해줘야 하지만 지금은 대충.. 나중에 정규식으로 바꿔주는게 좋을 것

In [183]:
 # #gotohere : 내부주소 - 스크롤 위치를 옮기는 주소 등임. 실제 페이지와는 연관이 없음.
urlparse('https://www.google.com/search#gotohere').fragment

'gotohere'

In [264]:
seed_list = []
seed_list.append(('https://comic.naver.com/webtoon/weekdayList?week=thu', 1))
# (url, 깊이)
visited_list = []
count = 1

while seed_list:
    seed = seed_list.pop(0) # Queue, FIFO => BFS
#     seed = seed_list.pop(-1) # Stack, LIFO => DFS : INF 루프에 빠질 수 있음. - 한 페이지에 대해 깊이 찾는 방식으로는 낫배드
    url_list = crawling(seed[0])
    
    print('{} {}'.format(count, seed)) # 원래는 몇번째 사이트, 몇번째 페이지 뭐 이런 정보들이 나와야만 함.
    
    if url_list:
        for _ in url_list:
            if _ not in [link[0] for link in visited_list]\
            and _ not in [link[0] for link in seed_list]:
                seed_list.append((_, seed[1]+1))

    print('찾은 갯수: {} 방문할 곳:{} 방문한 곳:{}'.format(
        len(url_list), len(seed_list), len(visited_list)))
    
    visited_list.append(seed)
    count += 1
    
    if seed[1] > 3:
        break
    
#     if count > 100:
#         break

1 ('https://comic.naver.com/webtoon/weekdayList?week=thu', 1)
찾은 갯수: 377 방문할 곳:216 방문한 곳:0
2 ('https://www.naver.com', 2)
찾은 갯수: 289 방문할 곳:437 방문한 곳:1
3 ('https://comic.naver.com/index', 2)
찾은 갯수: 170 방문할 곳:480 방문한 곳:2
4 ('https://novel.naver.com', 2)
찾은 갯수: 199 방문할 곳:656 방문한 곳:3
5 ('https://comic.naver.com/webtoon/list.nhn?titleId=772729', 2)
찾은 갯수: 122 방문할 곳:667 방문한 곳:4
6 ('https://comic.naver.com/webtoon/weekday', 2)
찾은 갯수: 1435 방문할 곳:1420 방문한 곳:5
7 ('https://comic.naver.com/genre/bestChallenge', 2)
찾은 갯수: 223 방문할 곳:1512 방문한 곳:6
8 ('https://comic.naver.com/genre/challenge', 2)
찾은 갯수: 225 방문할 곳:1607 방문한 곳:7
9 ('https://comic.naver.com/mypage/favorite', 2)
찾은 갯수: 0 방문할 곳:1606 방문한 곳:8
10 ('https://series.naver.com/comic/recommendList.nhn', 2)
찾은 갯수: 99 방문할 곳:1688 방문한 곳:9
11 ('https://series.naver.com/novel/recommendList.nhn', 2)
찾은 갯수: 127 방문할 곳:1755 방문한 곳:10
12 ('https://comic.naver.com/webtoon/genre', 2)
찾은 갯수: 987 방문할 곳:2098 방문한 곳:11
13 ('https://comic.naver.com/webtoon/creation', 2

TypeError: object of type 'NoneType' has no len()

In [265]:
urlparse('https://www.google.com/search?sfadfafd')

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='sfadfafd', fragment='')

### 15:00
#### 위키피디아와 같은 사이트들이 크롤링하기 굉장히 좋음.

---
### Scrapping
- 크롤링 vs scrapping
- 크롤링 : 기본 단위가 WWW임. 굉장히 큰 스케일이고,
    - 앞으로 이를 봇, 혹은 에이전트라고 부를 예정임.
    - parser / 우리의 편함을 위해 사용하는 것일뿐 실제 크롤러에 필수적인 존재는 아님. (정규식 사용하면 되므로)
- Scrapping : 보통은 원하는 데이터의 목적이 정해져있을 거이기 때문에 이걸 사용할 지도..<br><br>

- 크롤러 제작 시 depth를 만들어 둔 이유 : depth를 가지고 너무 멀리까지 가지 않도록!
- 기본적으로 Focused Crawling 전략은 세우기 마련...

### Page Rank
- 더 중요한 데이터가 존재할 수 있다. 예시로, 딥러닝에 관한 자료 중 논문, 저널 / 네이버 블로그의 중요성 차이.
- 이런 사이트에 대해 citation index 등 점수를 매긴 것이 존재 (얼마나 참조되었는 지를 기반)
- 이것을 실제 계산식으로 제공하는 것이 구글의 Page Rank!<br><br>

- 사람들이 많이 찾는 공인된 사이트에서 나온 자료를 더 높게 평가하는 것이 맞다.
- 많은 정보와 중요한 정보 사이에서 가중치를 다르게 줘야 할 것

모든 사이트들을 하나의노드라 생각하면, 노드와 노드 사이에는 방향성을 그릴 수 있다.<br>
내가 상대방 사이트로 링크를 걸고, (Outbound link)<br>
다른 사람들로부터 나에게 오는 링크를 Inbound link라고 하자.<br><br>

... 수렴할 때 까지 돌린다. (링크를 받는 사이트들이 영향력이 높아지게 된다)<br><br>

\* 예) 사이트의 검색 순위를 높이기 위해 미미한 새로운 사이트들을 여러 개 만들어<br>
높이려는 사이트를 일부로 링크를 주고 구글 검색 순위를 높인다.

In [ ]:
# Focused Crawling + Scraping의 예제 : 네이버 뉴스
시드 = 뉴스 헤드라인 (첫페이지)
while 리스트가 없을 때까지:
    특정 ID, CLASS 에서만 링크 수집 => Scraping

In [296]:
seed_list = list()
seed_list.append('https://news.naver.com/')

seen_list = list()

while seed_list:
    seed = seed_list.pop(0) #0이든지, -1이든지 ...
    
    resp = download(seed, headers=headers)
    
    # Scraping 영역 --> 
    if not re.search(r'text/html', 
                 resp.headers['Content-type'.lower()]):
        continue

    
    dom = BeautifulSoup(resp.text, 'lxml')
    
    # 헤드라인만 가져오게 됨.
    main_area = dom.select_one('#main_content') # 찾지 못할 시 None이 들어가게 됨.

    if not main_area:
        continue
    
    # li의 자식 중 a[href], li의 자식 중 div의 자식 중 a[href] : 이 2형태로 처리
    news_list = main_area.select('li > a[href], li > div > a[href]')

    article_area = dom.select_one('#articleBodyContents') # 이게 있으면 뉴스, 아니라면 건너뜀
    
    if article_area:
        file_name = re.sub(r'\W', '', seed.split('/')[-1]) # \W ASCII가 아닌 비표준 문자.
        
        with open('TXT/'+file_name+'.txt', 'w', encoding='utf-8') as f: # TXT 폴더에 저장
            f.write(article_area.text)
    # <-- Scraping 영역
    
    for link in news_list:
        link = urljoin(seed, link.attrs['href'])
        if link not in seed_list and link not in seen_list:
            seed_list.append(link)
            
    seen_list.append(seed)

InvalidSchema: No connection adapters were found for 'javascript:;'

In [ ]:
# <div id=main_content ...가 존재 시 첫 페이지.

In [ ]:
seed_list

In [ ]:
dom = BeautifulSoup(download(
    'https://news.naver.com', headers=headers).text, 'lxml')
len(dom.select('#main_content li > a[href], \
           #main_content li > div > a[href]'))

---
### ☆★DHTML (Dynamic HTML)
- 진짜 가져오고 싶은 데이터 : 보통 로그인 이후 가져오거나, 비동기적으로 서버에서 데이터를 불러옴.
- 일반적으로 클라이언트가 GET/POST로 request를 보내게 되고, 
- 그 후 서버에서 response를 보내주고 이를 클라이언트 웹브라우져가 렌더링해주는 방식임.<br><br>

- 클라이언트에서 데이터를 요청 보내고, 서버에서 데이터를 받는데 웹사이트에서 이동은 없는 경우.
- 예> http://brunch.co.kr 에서 검색을 진행하는 과정 : DHTML의 전형적인 예.
    - 소스보기에서 검색 결과를 찾으면 안나오고, Network 탭에 있는 xhr 파일에 데이터가 들어감.
    - AJAX를 활용한 DHTML의 전형적인 예..?

In [322]:
url = 'https://brunch.co.kr/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC'
resp = new_download(url)

False


In [323]:
dom = BeautifulSoup(resp.text, 'lxml')

In [325]:
dom.select('form')[0] # 이 form 태그를 통해 어디로 날라가는 지 확인할 수 있음

# 다만, 이런 JS가 익숙치 않으므로 chrome의 네트워크 탭을 활용해 이용하자.

<form action="/search" method="get">
<input autocomplete="off" id="searchQuery" name="q" placeholder="검색어를 입력해 주세요" title="검색어입력" type="text" value=""/>
<input name="profileId" type="hidden" value=""/>
</form>

In [300]:
# 크롬 개발자 도구 검사 상으로는 id="resultArticle"란에 있어야 하는데,
# 비어있음
dom

<!DOCTYPE HTML>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="width=1106" name="viewport"/>
<title>“시민이 화장실서 밥 먹는 나라” 서울대 청소노동자 사망에 분노한 시민들 : 네이버 뉴스</title>
<meta content="조선일보 : 네이버뉴스" property="me2:post_tag"/>
<meta content="조선일보" property="me2:category1"/>
<meta content="사회" property="me2:category2"/>
<meta content="https://imgnews.pstatic.net/image/023/2021/07/08/0003625253_001_20210708085002913.jpg" property="me2:image"/>
<meta content="“시민이 화장실서 밥 먹는 나라” 서울대 청소노동자 사망에 분노한 시민들" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://news.naver.com/main/read.naver?mode=LSD&amp;mid=sec&amp;oid=023&amp;aid=0003625253&amp;sid1=001" property="og:url"/>
<meta content="https://imgnews.pstatic.net/image/023/2021/07/08/0003625253_001_20210708085002913.jpg" property="og:image"/>
<meta content="서울대학교 청소노동자로 근무하던 50대 여성이 교내 휴게실에서 사망한 사건을 계기로 청소노동자

In [301]:
# Network 탭 - XHR 탭에 있는 파일에서 찾은 url
url = 'https://api.brunch.co.kr/v1/search/article?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&page=1&pageSize=20&highlighter=y&escape=y&sortBy=accu'
resp = new_download(url)

404 Client Error: Not Found for url: https://api.brunch.co.kr/robots.txt
False


In [303]:
# Content-Type : application/json
resp.headers

{'Date': 'Thu, 08 Jul 2021 07:32:07 GMT', 'Server': 'Apache', 'X-Apm-Svc-Key': 'dd780680-8eca-4183-a7ac-76bac94933d0', 'X-Apm-App-Name': 'API', 'X-Apm-Inst-Hash': '1349682606', 'Cache-Control': 'no-cache, no-store, must-revalidate, no-store', 'Pragma': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Access-Control-Allow-Methods': 'GET,POST,HEAD,OPTIONS,PUT,DELETE', 'Access-Control-Allow-Headers': 'Content-Type,X-Requested-With,accept,Origin,Access-Control-Request-Method,Access-Control-Request-Headers,brunch-client-os,X-CSRF-TOKEN,X-JSESSIONID,referer,brunch-agent', 'Access-Control-Allow-Credentials': 'true', 'Set-Cookie': 'bid=""; Domain=.brunch.co.kr; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/; Secure', 'Content-Language': 'ko-KR', 'Keep-Alive': 'timeout=15, max=14', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json;charset=UTF-8'}

In [311]:
for book in resp.json()['data']['list']:
    print(re.sub(r'</?[^>]+?>', '', book['title']))

프로그래밍 책을 처음 뗐습니다. - 모던 파이썬 입문 책떼기 후기
Python이 뭐야? 뱀이야? - 파이썬과 친해지기_01
비전공자도 개발자가 될 수 있을까? - 파이썬으로 시작하는 개발자 커리어
향후 금융권 주요 분석도구는 “엑셀+파이썬”이 된다.
14. 통계를 위한 파이썬 라이브러리, 넘파이 (2) - 파이썬으로 데이터 분석하기
12. 파일 읽고 쓰기 - 파이썬으로 데이터 분석하기
[인프런 리프] 파이썬 중급 강의4,5주 차후기
102. 파이썬 1주차- 파이썬 알아보기
[인프런 리프] 파이썬 중급 강의2주 차후기 - 클래스, 메소드 심화
파이썬 몰라도 크롤링 가능합니다 - 마우스 클릭 몇 번으로 경쟁사 후기 분석하기
금융권에 필요한 파이썬 스킬트리 #2.
연오의 파이썬, 두근두근 첫만남 - 연오의파이썬 I 지은이 박연오 I 출판사 인사이트
금융권에 필요한 파이썬 스킬트리 #1.
나만의 100일 프로젝트 완료 기록 - 파이썬과 영어공부
나만의 100일 프로젝트 절반의 기록 - 파이썬과 영어공부
124.(책 후기) 파이썬챌린지. 문제 중심
Python : 신조 - #파이썬 왈
데이콘, Fashion MNIST:의류 클래스 예측 - 파이썬 딥러닝 머신러닝 입문 - 4장 딥러닝 응용
파이썬 기초(1)
간절함과 목표 - 영어공부의 전부 그리고 노션 템플릿 - 영어공부 방법론과 파이썬 초보의 코딩 공부 - 2


In [ ]:
# 앞선 예시와 같이 AJAX를 이용해 비공개 데이터를 수집해오는 방식이 가능함.
# 로그인도 이와 같은 방식으로 해결됨! 
 # => 더미 아이디로 로그인하고 데이터 긁어오는 과정을 반복/자동화...

In [ ]:
# 로그인 예시 사이트 http://pythonscraping.com/pages/cookies/login.html

In [326]:
url = 'http://pythonscraping.com/pages/cookies/login.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'lxml')

In [329]:
# 원래는 code level에서 
form = dom.select_one('form')
input_list = dom.select('form input[name]')
# 등을 찾아 로그인 해야하는 지의 여부를 알아내야 함.

In [330]:
form.attrs

{'method': 'post', 'action': 'welcome.php'}

In [ ]:
urljoin(url, form.attrs['action']), form.attrs['method']

# 보내야하는 주소, 보내야하는 방법.

In [331]:
# 보내는 방법을 알기 위해 form input 확인.
for _ in input_list:
    print(_.attrs)

{'type': 'text', 'name': 'username'}
{'type': 'password', 'name': 'password'}
{'type': 'submit', 'value': 'Login'}


In [332]:
input_list = dom.select('form input[name]')

In [338]:
params = dict()
for _ in input_list:
    params[_.attrs['name']] = None

In [340]:
# 보내야 하는 parameter 목록
params

{'username': None, 'password': None}

In [343]:
account = [('test1', 'password'), ('test2', 'password')] # password는 따로 password라고 작성해야 함.

for _ in account:
    params['username'] = _[0]
    params['password'] = _[1]
    
    resp = download(urljoin(url, form.attrs['action']),
                    params=params, method=form.attrs['method'])
    
    print(resp.text)
    break


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


In [342]:
params # params dict에 값을 넣음

{'username': 'test1', 'password': 'password'}

In [ ]:
# 보통 로그인은 단발성 - 동시 접속자를 처리하기 위해 ~수초
# 이 로그인 값을 해당 사이트에서 관리할 수 있는 방법으로 저장하게 됨.
# 예시> 쿠키 - 클라이언트에 저장, 세션 - 서버에 저장.
# 
# 쿠키를 이용해 로그인 값을 찾아내고, 그 값을 이용해 로그인 해보는 방법.

In [344]:
# 이제 개발자 도구 탭에서 네트워크 말고 Application-Storage-Cookies
# username과 loggedin을 찾을 수 있음
#
# 우리가 코드에서 접근을 할 때는 이 쿠키값을 만들지 못해서 
# 정상적으로 로그인되지 않음
# 이 쿠키를 관리해주는 걸 import
from requests.sessions import Session

In [345]:
# 쿠키 보관, 연결 유지 등..
session = Session()

In [347]:
session.cookies.get_dict()

{}

In [356]:
resp = session.request(url=urljoin(url, form.attrs['action']), 
                       data=params, method=form.attrs['method'])

In [357]:
resp.text # 로그인 실패

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [358]:
session.cookies.get_dict() # 되지만 쿠키가 생성되었음.

{'loggedin': '1', 'username': 'test1'}

In [359]:
session.request(url=urljoin(url, form.attrs['action']), # 쿠키를 이용해 parameter 없이 로그인 시도 - 성공함.
               method=form.attrs['method']).text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [360]:
# 물론 사이트마다 이전 쿠키를 허용해주느냐에 대한 문제 등이 있지만
# 대다수에 사이트에서는 쿠키가 있으면 로그인할 수 있음.

In [361]:
session.cookies.clear()

In [362]:
session.request(url=urljoin(url, form.attrs['action']),
               method=form.attrs['method']).text
# 쿠키가 없으니 당연히 로그인 안됨.

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [365]:
# 임의로 쿠키 값을 넣어주자. JSON
import json
test_cookie = {'loggedin': '1', 'username': 'test1'}
with open('test_cookie.txt', 'w') as f:
    json.dump(test_cookie, f)

In [366]:
with open('test_cookie.txt', 'r') as f:
    cookie = json.load(f)
# 로그인해서 쿠키값만 저장했다고 가정하는 것

In [367]:
cookie

{'loggedin': '1', 'username': 'test1'}

In [368]:
for k, v in cookie.items():
    session.cookies.set(k, v)

In [370]:
session.cookies.get_dict()

{'loggedin': '1', 'username': 'test1'}

In [371]:
# 다시 로그인 정보(params) 없이 쿠키만 있는 채로 로그인 시도
session.request(url=urljoin(url, form.attrs['action']),
               method=form.attrs['method']).text

# 잘됨

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [372]:
# 쿠키 값 응용 : 
request(url=urljoin(url, form.attrs['action']),
            method=form.attrs['method'],
            cookies=session.cookies).text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [373]:
# 예시) 네이버에 로그인
resp = download('https://www.naver.com')
dom = BeautifulSoup(resp.text, 'lxml')

In [375]:
dom.select('form') # 검색 페이지밖에 안나옴.. 로그인 페이지가 별도로 있을 예정

[<form action="https://search.naver.com/search.naver" id="sform" method="get" name="sform" role="search">
 <fieldset>
 <legend class="blind">검색</legend>
 <select class="blind" id="where" name="where" title="검색 범위 선택">
 <option selected="selected" value="nexearch">통합검색</option><option value="post">블로그</option><option value="cafeblog">카페</option><option value="cafe">- 카페명</option><option value="article">- 카페글</option><option value="kin">지식iN</option><option value="news">뉴스</option><option value="web">사이트</option><option value="category">- 카테고리</option><option value="site">- 사이트</option><option value="movie">영화</option><option value="webkr">웹문서</option><option value="dic">사전</option><option value="100">- 백과사전</option><option value="endic">- 영어사전</option><option value="eedic">- 영영사전</option><option value="krdic">- 국어사전</option><option value="jpdic">- 일본어사전</option><option value="hanja">- 한자사전</option><option value="terms">- 용어사전</option><option value="book">책</option><option value="music">

In [376]:
resp = download('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')
dom = BeautifulSoup(resp.text, 'lxml')

In [378]:
form = dom.select_one('form')
input_list = dom.select('input[name]')

In [379]:
form.attrs['action'], form.attrs['method']

('https://nid.naver.com/nidlogin.login', 'POST')

In [384]:
for _ in input_list:
    print(_.attrs['name'], 
          _.attrs['value'] if _.has_attr('value') else '입력필요')

localechange 
dynamicKey RoNCXlX-461kn5LVOBoO19G9zO1H1GTmyGgeyX9TtpLP3XVsFXmNpc5VQaj5rg-beSB5QlPjhv9Vv9quGPfQu_HU0mIM6RReAX5-hkaa1Hw
encpw 
enctp 1
svctype 1
smart_LEVEL 1
bvsd 
encnm 
locale ko_KR
url https://www.naver.com
id 
pw 입력필요
nvlong off
nclicks_nsc nid.login_kr
nid_buk none
ncaptchaSplit none
id_error_msg 아이디를 입력해주세요.
pw_error_msg 비밀번호를 입력해주세요.
locale ko_KR
adult_surl_v2 


In [400]:
test_cookie = {'id': 'test2', 'pw': 'test1'}
with open('test_cookie.txt', 'w') as f:
    json.dump(test_cookie, f)

In [401]:
with open('test_cookie.txt', 'r') as f:
    cookie = json.load(f)

In [402]:
cookie

{'id': 'test2', 'pw': 'test1'}

In [403]:
for k, v in cookie.items():
    session.cookies.set(k, v)

In [404]:
request(url='https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com',
            method=form.attrs['method'],
            cookies=session.cookies).text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t<meta charset="UTF-8">\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">\r\n\t<meta property="og:type" content="website">\r\n\t<meta property="og:title" content="[네이버: 로그인]">\r\n\t<meta property="og:description" content="안전한 로그인을 위해 주소창의 URL과 자물쇠 마크를 확인하세요!">\r\n\t<meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png">\r\n\t<meta property="og:image:type" content="image/png">\r\n\t<meta property="og:image:width" content="1200">\r\n\t<meta property="og:image:height" content="1200">\t\t\r\n\t<title>네이버 : 로그인</title>\r\n\t<link rel="stylesheet" type="text/css" href="/login/css/global/desktop/w_20210422.css?20210422">\r\n</head>\r\n<body class="">\r\n<div class="theme_txt" id="theme_txt_message">\r\n\t<p><strong>안전한 네이버 로그인을 위해 주소창의 URL과 자물쇠 마크를 확인하세요!</strong></p>

In [ ]:
# 암호화 과정 등을 거치기 때문에 안되지만
# 이를 해결하기 위한 방법 : 로그인 상태 유지를 체크 한 후 로그인 할 시 쿠키가 남게 됨
# 이를 활용

In [416]:
naver = [
    {
        'name':'nid_enctp',
        'value':'1',
        'domain':'.nid.naver.com',
        'path':'/''
    },
    {
        'name':'nid_buk',
        'value':'NR3UON6EWA',
        'domain':'.nid.naver.com',
        'path':'/'
    },
    {
        'name':'NID_SES',
        'value':'AAABkBbH6/IZTIoZm+YdtjpakLWDf2j1AgISufgiRB5IA7/m8xIqSp3D7Q/7sB3dvQfimZmW/vRlOnt2N4P63lKNDPla/UqOTpw7lmECD/8Z34hR9DwPDeRX3i8oklXBANaAdkq0Wqc8E0YNbtufSrUkxX1RtrGPp53uH44hEMRhdv//dzJPbLsjq0uVejxEg8FyfqLb2Fwx8uHQ3+DI4RYepHPpAKE7Sm5nhEpnW4ijgl/Uut2P6/pMDpqIgAj2TiHiMIMmGAs9jNLiSDAevr/k6rctiw+jl7cxACwJUafS+iYvfPI4KRbuNqdwgIy3eGDeKYiv8I8EBHpKPb/RjdM2yKVqWzt7vqvflj4pQtTRFTCk0dC/4pZpilP6GFrtpHICCspYXSDGKk8SeI9j3i6Sv7+VHC/hy0KpxYc04hkdq/BEngisglpmiC0G7VYZ1XbJO/SpMj',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'NID_JKL',
        'value':'9S2/pCZJ+tsWgscTxIwOQeqZBD8Wz8=',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'NID_AUT',
        'value':'rLWcvaoZbz3KiKZLWppqyIRnxD0teFHenKnCEYUXYkS067Bv+8Dk',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'nid_inf',
        'value':'438370032',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'page_uid',
        'value':'hMi+YspssfOyKVlssssssI0-05896',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'nid_slevel',
        'value':'-1',
        'domain':'.nid.naver.com',
        'path':'/'
    },
    {
        'name':'NM_THEME_EDIT',
        'value':'',
        'domain':'www.naver.com',
        'path':'/'
    },
    {
        'name':'NFS',
        'value':'2',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'_ga_4BKHBFKFK0',
        'value':'GS1.1.16283126.1.1.1683136.50',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'NV_WETR_LAST_ACCESS_RGN_M',
        'value':'"MDkTAxMjU="',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'_fbp',
        'value':'fb.1.1629276.910894030=',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'_ga',
        'value':'GA1.2.8882.1621083127',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'MM_NEW',
        'value':'1',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'nx_ssl',
        'value':'2',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'NM_THEME_LAST_FIXED',
        'value':'LIVING',
        'domain':'www.naver.com',
        'path':'/'
    },
    {
        'name':'NNB',
        'value':'NR3NUUON65EWA',
        'domain':'.naver.com',
        'path':'/'
    },
    {
        'name':'PM_CK_loc',
        'value':'5b3cfba8eee6deb5268c1cf9cf2c843f5c4d5e',
        'domain':'www.naver.com',
        'path':'/'
    },
    {
        'name':'NV_WETR_LOCATION_RGN_M',
        'value':'"MDkyOTAxMjU="',
        'domain':'.naver.com',
        'path':'/'
    },
]

In [433]:
session.cookies.clear()

for i, _ in enumerate(naver):
    session.cookies.set(naver[i]['name'], naver[i]['value'],
                       domain=naver[i]['domain'], path=naver[i]['path'])

In [436]:
# 예시 코드
# for i in [i for i, _ in enumerate(naver) if _ == 'Medium']:
#     e = i
# #     print(naver[s:e+1])
#     session.cookies.set(naver[s], naver[s+1],
#                        domain=naver[s+2], path=naver[s+3])
#     s = e+1

In [434]:
# 네이버 메일 : 로그인하라고 뜸
resp = download('https://mail.naver.com')
resp.text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t<meta charset="UTF-8">\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">\r\n\t<meta property="og:type" content="website">\r\n\t<meta property="og:title" content="[네이버: 로그인]">\r\n\t<meta property="og:description" content="안전한 로그인을 위해 주소창의 URL과 자물쇠 마크를 확인하세요!">\r\n\t<meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png">\r\n\t<meta property="og:image:type" content="image/png">\r\n\t<meta property="og:image:width" content="1200">\r\n\t<meta property="og:image:height" content="1200">\t\t\r\n\t<title>네이버 : 로그인</title>\r\n\t<link rel="stylesheet" type="text/css" href="/login/css/global/desktop/w_20210422.css?20210422">\r\n</head>\r\n<body class="">\r\n<div class="theme_txt" id="theme_txt_message">\r\n\t<p><strong>안전한 네이버 로그인을 위해 주소창의 URL과 자물쇠 마크를 확인하세요!</strong></p>

In [ ]:
# 메일 화면이 뜨면서 로그인 된 것을 확인할 수 있음
resp = session.get('https://mail.naver.com')
resp.text